In [6]:
#Step 1 : Select kernet, Pyhton 3.8 which ever that is used in the environent

In [7]:
import os
import json
import pandas as pd
import traceback

In [8]:
from langchain.chat_models import ChatOpenAI

In [9]:
#from dotenv import load_dotenv
#load_dotenv()
#KEY=os.getenv("OPENAI_API_KEY")
KEY="sk-jey3RfzWi3WspUjs74kcT3B1bkFJsrJIcdRXSAABsy8rFFpj"

In [10]:
llm=ChatOpenAI(openai_api_key=KEY,model="gpt-3.5-turbo",temperature=0.5)

In [11]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fda40c6f850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fda5080d610>, temperature=0.5, openai_api_key='sk-jey3RfzWi3WspUjs74kcT3B1bkFJsrJIcdRXSAABsy8rFFpj', openai_proxy='')

In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
#Design an input prompt that is provided to an LLM
#Design an output prompt

In [32]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer"
    }
}

In [13]:
TEMPLATE='''
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure the format your response like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
'''

#tone is difficult,intermediate,easy

In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)
quiz_generation_prompt

PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure the format your response like RESPONSE_JSON below and use it as a guide.Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n')

In [17]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure the format your response like RESPONSE_JSON below and use it as a guide.Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fda40c6f850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fda5080d610>, temperature=0.5, openai_api_key='sk-jey3RfzWi3WspUjs74kcT3B1bkFJsrJIcdRXSAABsy8rFFpj', openai_proxy=''), output_key='quiz')

In [18]:
#Evaluation the first template
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple choice quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity.
If the quiz is not at per with the congnitive and analytical abilities of the students ,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert Englist Writer of the above quiz:
"""

In [19]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE
)

In [21]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt,output_key="review",verbose=True)
review_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure the format your response like RESPONSE_JSON below and use it as a guide.Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fda40c6f850>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fda5080d610>, temperature=0.5, openai_api_key='sk-jey3RfzWi3WspUjs74kcT3B1bkFJsrJIcdRXSAABsy8rFFpj', openai_proxy=''), output_key='review')

In [22]:
generate_evaluate_chain=SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [27]:
file_path="/Users/suchitra/suchitra/genai-learning/mcqgen/data.txt"
file_path

'/Users/suchitra/suchitra/genai-learning/mcqgen/data.txt'

In [28]:
with open(file_path,'r') as file:
    TEXT = file.read()
    

In [29]:
TEXT

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nMachine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupe

In [33]:
#Serialize the python dictornary to json formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [34]:
NUMBER=5
SUBJECT="machine learning"
TONE="simple"

In [ ]:
#how to setup token using tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )

    #output will be in response
    

In [36]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost: 0.0


In [ ]:
#Get the response
quiz = response.get("quiz")
quiz = json.loads(quiz)

In [ ]:
quiz_table_data=[]
for key,value in quiz.items():
    mcq=value["mcq"]
    options= " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    

In [ ]:
quiz = pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False)

#Output
#MCQ,Choices,Correct
#Who coined the term macine learning?,a: Donald Hebb | b:Arthur Samuel | c: Walter Pitts | d: Warren McCulloch, b
#What was the earliest machine learning model introduced by Arthur Samuel?, a: Speech Recognition | b: Image classification | c: Word Recognition | d: image Recognition, a
#Whict book introduced the Hebbian theory?
#4th question similar to above format
#5th question similar to above format
